In [1]:
import importlib
import preprocessing
importlib.reload(preprocessing)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thiba\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thiba\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'preprocessing' from 'd:\\Travail\\ESILV\\S9\\NLP\\project_2\\Streamlit_NLP\\preprocessing.py'>

# Preprocessing

In [2]:
import os
import pandas as pd

def concat_data_files():
    folder_path = './data'
    xlsx_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]


    data_frames = []
    for file in xlsx_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)
        data_frames.append(df)
    df = pd.concat(data_frames, ignore_index=True)
    df.to_csv('./cleaned_data/concat_data.csv', index=False)

concat_data_files()

## Handling missing values, duplicate and translation

In [3]:
df = pd.read_csv('./cleaned_data/concat_data.csv',delimiter=',')
df.isnull().sum()

note                10331
auteur                  1
avis                    0
assureur                0
produit                 0
type                    0
date_publication        0
date_exp                0
avis_en                 2
avis_cor            34000
avis_cor_en         34004
dtype: int64

In [4]:
df['auteur'] = df['auteur'].fillna('Inconnu')
df = df.dropna(subset=['note'])
from deep_translator import GoogleTranslator

avis_to_translate = df[df['avis_en'].isnull()]

def translate_avis(avis):
    return GoogleTranslator(source='fr', target='en').translate(avis)

df.loc[df['avis_en'].isnull(), 'avis_en'] = avis_to_translate['avis'].apply(translate_avis)

# Vérifier les premières lignes après la traduction
df.head(10)


,note,auteur,avis,assureur,produit,type,date_publication,date_exp,avis_en,avis_cor,avis_cor_en
0,4.0,audurier-c-136272,La personne au téléphone était Clair et sympat...,L'olivier Assurance,auto,train,06/10/2021,01/10/2021,The person on the phone was clear and friendly...,NaN,NaN
1,4.0,paul-a-122970,"Satisfait.\n\nRéactivité, simplicité. Prix att...",APRIL Moto,moto,train,09/07/2021,01/07/2021,"Satisfied.\n\nReactivity, simplicity. Attracti...",NaN,NaN
2,1.0,kitty-38517,"Assureur à fuir, n assure pas ses responsabili...",SwissLife,vie,train,15/10/2020,01/10/2020,"Insurer to flee, does not ensure its responsib...",NaN,NaN
3,1.0,laure97134-87907,Voilà 3 mois que la GMF me fait attendre pour ...,GMF,habitation,train,03/03/2020,01/03/2020,The GMF has been waiting for a water damage fo...,NaN,NaN
4,3.0,bourouane-l-129916,Je suis bien avec cet assurance.elle est prati...,L'olivier Assurance,auto,train,28/08/2021,01/08/2021,I am good with this insurance. She is practica...,NaN,NaN
5,5.0,ilyas-k-115898,Le drive box nest pas tres fiable.Ca demande b...,Direct Assurance,auto,train,03/06/2021,01/06/2021,The Drive Box is not very reliable.ca requests...,NaN,NaN
6,5.0,nadette-121889,J'ai malheureusement eu à faire appel à plusie...,Pacifica,auto,train,30/06/2021,01/06/2021,"Unfortunately, I had to call on Pacifica for b...",NaN,NaN
7,4.0,mimile-81147,Suite à mon bris de glace le traitement de cel...,Pacifica,auto,train,20/11/2019,01/11/2019,Following my ice broken the treatment of it wa...,NaN,NaN
8,1.0,rahma-63689,"Bonjour , Actuellement je suis chez Active Ass...",Active Assurances,auto,train,01/05/2018,01/05/2018,"Hello, currently I am at Active Insurance unde...",NaN,NaN
9,1.0,bichon-70941,Ma mère est décédée depuis plusieurs mois et ...,SwissLife,vie,train,04/02/2019,01/02/2019,My mother has died for several months and sti...,NaN,NaN


In [5]:
df.isnull().sum()

note                    0
auteur                  0
avis                    0
assureur                0
produit                 0
type                    0
date_publication        0
date_exp                0
avis_en                 0
avis_cor            24104
avis_cor_en         24104
dtype: int64

In [6]:
duplicates_count = df.duplicated().sum()
print(f"Nombre de doublons dans le DataFrame : {duplicates_count}")
df = df.drop_duplicates()

Nombre de doublons dans le DataFrame : 4


## Correction Spelling

In [7]:
import re
from symspellpy import SymSpell, Verbosity
import pandas as pd

# Initialisation de SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Charger un dictionnaire de fréquence
sym_spell.load_dictionary("./dictionaries/frequency_dictionary_en_82_765.txt", term_index=0, count_index=1)

def clean_text(text):
    """
    Fonction pour nettoyer le texte avant la correction.
    """
    text = text.strip()
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def correct_spelling_fast(text, lang='en'):
    """
    Corrige l'orthographe rapidement en anglais.
    Utilise SymSpell pour la correction rapide.
    """
    # Découpe le texte en mots
    words = text.split()

    corrected_words = []
    for word in words:
        # Utilisation de SymSpell pour la correction des fautes
        correction = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        # Si une correction est trouvée, utiliser la première suggestion
        corrected_words.append(correction[0].term if correction else word)

    # Recombine les mots corrigés en une phrase
    corrected_text = ' '.join(corrected_words)
    return corrected_text

# Appliquer la correction sur la colonne 'avis' pour le français (ou anglais selon le dictionnaire)
#df['avis_cor'] = df['avis'].apply(lambda x: correct_spelling_fast(clean_text(x), lang='fr'))

df['avis_cor_en'] = df['avis_en'].apply(lambda x: correct_spelling_fast(clean_text(x), lang='en'))
df.to_csv('./cleaned_data/cleaned_rev.csv', index=False)

In [8]:

# Initialisation de SymSpell
sym_spell_fr = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Charger un dictionnaire de fréquence
sym_spell_fr.load_dictionary("./dictionaries/frequency.txt", term_index=0, count_index=1)

def clean_text(text):
    """
    Fonction pour nettoyer le texte avant la correction.
    """
    text = text.strip()
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def correct_spelling_fast_fr(text, lang='en'):
    """
    Corrige l'orthographe rapidement en français
    Utilise SymSpell pour la correction rapide.
    """
    # Découpe le texte en mots
    words = text.split()

    corrected_words = []
    for word in words:#
        # Utilisation de SymSpell pour la correction des fautes
        correction = sym_spell_fr.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        # Si une correction est trouvée, utiliser la première suggestion
        corrected_words.append(correction[0].term if correction else word)

    # Recombine les mots corrigés en une phrase
    corrected_text = ' '.join(corrected_words)
    return corrected_text

# Appliquer la correction sur la colonne 'avis' pour le français (ou anglais selon le dictionnaire)
df['avis_cor'] = df['avis'].apply(lambda x: correct_spelling_fast(clean_text(x), lang='fr'))

#df['avis_cor_en'] = df['avis_en'].apply(lambda x: correct_spelling_fast(clean_text(x), lang='en'))
df.to_csv('./cleaned_data/cleaned_rev.csv', index=False)

# Tests for additional functions

In [9]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

def topic_modeling(df, column, n_topics=5):
    # Vectorisation des avis (bag-of-words)
    vectorizer = CountVectorizer(stop_words='english')
    X = vectorizer.fit_transform(df[column])

    # Appliquer LDA
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(X)
    topics_words = []
    # Affichage des termes les plus fréquents pour chaque sujet
    terms = vectorizer.get_feature_names_out()
    for idx, topic in enumerate(lda.components_):
        print(f"Topic {idx+1}:")
        print([terms[i] for i in topic.argsort()[-10:]])  # Afficher les 10 termes les plus fréquents du sujet
        topics_words.append([terms[i] for i in topic.argsort()[-10:]])
        print()

    return lda, topics_words

lda_model, tw = topic_modeling(df, 'avis_cor_en', n_topics=5)
print(tw)

Topic 1:
['documents', 'sent', 'service', 'phone', 'email', 'file', 'months', 'mutual', 'insurance', 'contract']

Topic 2:
['green', 'just', 'vehicle', 'time', 'increase', 'years', 'card', 'contract', 'year', 'insurance']

Topic 3:
['home', 'car', 'care', 'does', 'disaster', 'years', 'expert', 'vehicle', 'mail', 'insurance']

Topic 4:
['simple', 'fast', 'thank', 'recommend', 'good', 'prices', 'price', 'service', 'insurance', 'satisfied']

Topic 5:
['insured', 'year', 'mail', 'responsible', 'vehicle', 'euros', 'accident', 'car', 'insurance', 'years']

[['documents', 'sent', 'service', 'phone', 'email', 'file', 'months', 'mutual', 'insurance', 'contract'], ['green', 'just', 'vehicle', 'time', 'increase', 'years', 'card', 'contract', 'year', 'insurance'], ['home', 'car', 'care', 'does', 'disaster', 'years', 'expert', 'vehicle', 'mail', 'insurance'], ['simple', 'fast', 'thank', 'recommend', 'good', 'prices', 'price', 'service', 'insurance', 'satisfied'], ['insured', 'year', 'mail', 'respon

In [10]:
import json
file_path = './cleaned_data/topics_word.json'

# Enregistrer sous format JSON
with open(file_path, 'w') as json_file:
    json.dump(tw, json_file, indent=4)

print(f"File saved as {file_path}")

File saved as ./cleaned_data/topics_word.json


In [11]:
topics = {
    "Topic 1": "Insurance Documents & Service Communication",
    "Topic 2": "Insurance Pricing & Contract Renewal",
    "Topic 3": "Vehicle & Home Insurance Claims",
    "Topic 4": "Affordable & Efficient Insurance Service",
    "Topic 5": "Insurance Responsibility & Vehicle Coverage"
}

# Chemin du fichier à enregistrer
file_path = './cleaned_data/topics_name.json'

# Enregistrer sous format JSON
with open(file_path, 'w') as json_file:
    json.dump(topics, json_file, indent=4)

print(f"File saved as {file_path}")

File saved as ./cleaned_data/topics_name.json


In [23]:
with open('./cleaned_data/topics_name.json', 'r') as f:
    topics_name = json.load(f)
print(topics_name.values())

dict_values(['Insurance Documents & Service Communication', 'Insurance Pricing & Contract Renewal', 'Vehicle & Home Insurance Claims', 'Affordable & Efficient Insurance Service', 'Insurance Responsibility & Vehicle Coverage'])


In [ ]:
from transformers import pipeline
import json

def call_zero_shot_topic(input_rev):
    with open('./cleaned_data/topics_word.json', 'r') as f:
        topics_word = json.load(f)

    with open('./cleaned_data/topics_name.json', 'r') as f:
        topics_name = json.load(f)

    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    labels = list(topics_name.values())
    result = classifier(input_rev, candidate_labels=labels)

text = 'the person on the phone was clear and friendly she explained everything to me i would recommend your insurance without hesitation thank you to the person i contacted'
call_zero_shot_topic(text)

# Model

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

def prepare_data():
    df=pd.read_csv('./cleaned_data/cleaned_rev.csv',delimiter=',')
    df = df[['note','avis_cor_en']]
    df['note'] = df['note'].astype(int)
    df['note'] = df['note'] - 1
    x_train, x_test, y_train, y_test = train_test_split(df['avis_cor_en'], df['note'], test_size=0.2)
    
    train_data = Dataset.from_dict({
    'text': x_train.astype(str).tolist(),
    'label': y_train.astype(int).tolist()
    })
    test_data = Dataset.from_dict({
        'text': x_test.astype(str).tolist(),
        'label': y_test.astype(int).tolist()
    })
    return train_data, test_data

def model_setup(model,tokenizer):
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)
    
    train_data, test_data = prepare_data()
    train_data = train_data.map(tokenize_function, batched=True)
    test_data = test_data.map(tokenize_function, batched=True)

    train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=4,   # batch size for training
    per_device_eval_batch_size=4,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    )

    trainer = Trainer(
    model=model,                         # the model to train
    args=training_args,                  # training arguments
    train_dataset=train_data,            # training dataset
    eval_dataset=test_data               # evaluation dataset
    )

    # Train the model
    trainer.train()


c:\Users\thiba\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
torch.cuda.empty_cache()

In [15]:
model_setup(model,tokenizer)

  0%|          | 13/14460 [00:01<17:49, 13.50it/s] 

{'loss': 1.6064, 'grad_norm': 3.4171457290649414, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 23/14460 [00:01<15:40, 15.35it/s]

{'loss': 1.6003, 'grad_norm': 3.4988625049591064, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


  0%|          | 33/14460 [00:02<15:28, 15.54it/s]

...

{'loss': 0.6524, 'grad_norm': 19.005878448486328, 'learning_rate': 3.581661891117479e-08, 'epoch': 3.0}


100%|██████████| 14460/14460 [17:57<00:00, 15.91it/s]

{'loss': 1.0863, 'grad_norm': 13.998098373413086, 'learning_rate': 0.0, 'epoch': 3.0}


100%|██████████| 14460/14460 [18:03<00:00, 13.34it/s]

{'train_runtime': 1083.7298, 'train_samples_per_second': 53.371, 'train_steps_per_second': 13.343, 'train_loss': 0.9798151904946376, 'epoch': 3.0}


In [16]:
model.save_pretrained("model_saved")
tokenizer.save_pretrained("model_saved")

('model_saved\\tokenizer_config.json',
 'model_saved\\special_tokens_map.json',
 'model_saved\\vocab.txt',
 'model_saved\\added_tokens.json')

In [17]:
df_test=pd.read_csv('./cleaned_data/cleaned_rev.csv',delimiter=',')
df_test = df_test[['note','avis_cor_en']]
df_test['note'] = df_test['note'].astype(int)

In [18]:
df['note'] = df['note'].round().astype(int)